In [ ]:
"""
TITLE - DISEASE PREDICTION AND MEDICINE RECOMMENDATAIONS SYSTEM

Features:
- Random Forest + SVM + Gradient Boosting models
- Ensemble voting for better accuracy
- Model comparison and confidence analysis
- Detailed explanations for predictions
- Comprehensive medicine recommendations
"""


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import joblib
import warnings
warnings.filterwarnings('ignore')

# Import medicine database
try:
    from medicine_database import DISEASE_MEDICINE_DATABASE
    MEDICINE_DB_LOADED = True
except ImportError:
    print("⚠️  medicine_database.py not found. Medicine recommendations will be limited.")
    MEDICINE_DB_LOADED = False
    DISEASE_MEDICINE_DATABASE = {}


class MultiModelDiseasePrediction:
    def __init__(self):
        self.model_rf = None
        self.model_svm = None
        self.model_gb = None
        self.ensemble_model = None
        self.symptom_list = []
        self.symptom_to_original = {}
        self.disease_symptom_map = {}
        self.confidence_threshold = 0.40
        self.model_accuracies = {}

    def print_header(self, title, symbol="═"):
        """Print formatted header"""
        width = 80
        print("\n" + symbol * width)
        print(f"{title:^{width}}")
        print(symbol * width)

    def print_section(self, title):
        """Print section divider"""
        print(f"\n{'─' * 80}")
        print(f"  {title}")
        print('─' * 80)

    def load_data(self):
        """Load datasets"""
        try:
            self.print_header("📂 LOADING DATASETS", "═")

            print("\n⏳ Loading Training.csv...", end=" ")
            self.training_data = pd.read_csv('\Datasets\Training.csv')
            print(f"✅ [{self.training_data.shape[0]} records]")

            print("⏳ Loading symptom_Description.csv...", end=" ")
            self.disease_descriptions = pd.read_csv('\Datasets\symptom_Description.csv')
            print(f"✅ [{len(self.disease_descriptions)} diseases]")

            print("⏳ Loading symptom_precaution.csv...", end=" ")
            self.disease_precautions = pd.read_csv('\Datasets\symptom_precaution.csv')
            print(f"✅ [{len(self.disease_precautions)} precautions]")

            print("\n✅ All required datasets loaded successfully!")
            return True

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: {str(e)}")
            return False

    def preprocess_data(self):
        """Preprocess data and build symptom-disease mapping"""
        self.print_header("🔄 PREPROCESSING DATA", "═")

        disease_col = None
        for col in ['prognosis', 'Disease', 'disease', 'label']:
            if col in self.training_data.columns:
                disease_col = col
                break

        if disease_col is None:
            print("\n❌ ERROR: Disease column not found!")
            return None, None

        symptom_cols = [col for col in self.training_data.columns
                       if col != disease_col and not col.startswith('Unnamed')]

        for col in symptom_cols:
            cleaned = col.replace('_', ' ').strip().lower()
            self.symptom_to_original[cleaned] = col
            self.symptom_list.append(cleaned)

        self.symptom_list = sorted(self.symptom_list)

        # Build disease-symptom mapping
        for _, row in self.training_data.iterrows():
            disease = row[disease_col]
            symptoms = []

            for col in symptom_cols:
                if row[col] == 1:
                    symptom_name = col.replace('_', ' ').strip().lower()
                    symptoms.append(symptom_name)

            if disease not in self.disease_symptom_map:
                self.disease_symptom_map[disease] = set()
            self.disease_symptom_map[disease].update(symptoms)

        print(f"\n✅ Symptoms extracted: {len(self.symptom_list)}")
        print(f"✅ Diseases in database: {len(self.disease_symptom_map)}")

        original_cols = [self.symptom_to_original[s] for s in self.symptom_list]
        self.X = self.training_data[original_cols].values
        self.y = self.training_data[disease_col].values

        print(f"✅ Training matrix: {self.X.shape}")

        return self.X, self.y

    def train_models(self):
        """Train multiple ML models and create ensemble"""
        self.print_header("🤖 TRAINING MULTIPLE ML MODELS", "═")

        X_train, X_test, y_train, y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=42, stratify=self.y
        )

        print("\n" + "="*80)
        print("MODEL 1: RANDOM FOREST CLASSIFIER".center(80))
        print("="*80)
        print("⏳ Training Random Forest...")

        self.model_rf = RandomForestClassifier(
            n_estimators=200,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1,
            class_weight='balanced'
        )

        self.model_rf.fit(X_train, y_train)
        y_pred_rf = self.model_rf.predict(X_test)
        acc_rf = accuracy_score(y_test, y_pred_rf)
        self.model_accuracies['Random Forest'] = acc_rf

        print(f"✅ Random Forest Trained!")
        print(f"📊 Accuracy: {acc_rf*100:.2f}%")
        print(f"🎯 Strengths: Handles complex patterns, good with binary features")

        print("\n" + "="*80)
        print("MODEL 2: SUPPORT VECTOR MACHINE (SVM)".center(80))
        print("="*80)
        print("⏳ Training SVM (this may take a moment)...")

        self.model_svm = SVC(
            kernel='rbf',
            C=10,
            gamma='scale',
            probability=True,
            random_state=42,
            class_weight='balanced'
        )

        self.model_svm.fit(X_train, y_train)
        y_pred_svm = self.model_svm.predict(X_test)
        acc_svm = accuracy_score(y_test, y_pred_svm)
        self.model_accuracies['SVM'] = acc_svm

        print(f"✅ SVM Trained!")
        print(f"📊 Accuracy: {acc_svm*100:.2f}%")
        print(f"🎯 Strengths: Excellent for high-dimensional data, robust")

        print("\n" + "="*80)
        print("MODEL 3: GRADIENT BOOSTING CLASSIFIER".center(80))
        print("="*80)
        print("⏳ Training Gradient Boosting...")

        self.model_gb = GradientBoostingClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=42,
            subsample=0.8
        )

        self.model_gb.fit(X_train, y_train)
        y_pred_gb = self.model_gb.predict(X_test)
        acc_gb = accuracy_score(y_test, y_pred_gb)
        self.model_accuracies['Gradient Boosting'] = acc_gb

        print(f"✅ Gradient Boosting Trained!")
        print(f"📊 Accuracy: {acc_gb*100:.2f}%")
        print(f"🎯 Strengths: Sequential learning, corrects mistakes iteratively")

        print("\n" + "="*80)
        print("ENSEMBLE MODEL: VOTING CLASSIFIER".center(80))
        print("="*80)
        print("⏳ Creating Ensemble (Combining all 3 models)...")

        self.ensemble_model = VotingClassifier(
            estimators=[
                ('rf', self.model_rf),
                ('svm', self.model_svm),
                ('gb', self.model_gb)
            ],
            voting='soft',  # Uses probability averaging
            weights=[2, 1, 1]  # Give more weight to Random Forest
        )

        self.ensemble_model.fit(X_train, y_train)
        y_pred_ensemble = self.ensemble_model.predict(X_test)
        acc_ensemble = accuracy_score(y_test, y_pred_ensemble)
        self.model_accuracies['Ensemble'] = acc_ensemble

        print(f"✅ Ensemble Model Created!")
        print(f"📊 Accuracy: {acc_ensemble*100:.2f}%")
        print(f"🎯 Strengths: Combines strengths of all models for best results")

        # Display comparison
        print("\n" + "="*80)
        print("📊 MODEL PERFORMANCE COMPARISON".center(80))
        print("="*80)
        print(f"\n{'Model':<25} {'Accuracy':<15} {'Performance'}")
        print("-" * 80)

        for model_name, acc in sorted(self.model_accuracies.items(), key=lambda x: x[1], reverse=True):
            bar_length = int(acc * 50)
            bar = '█' * bar_length + '░' * (50 - bar_length)

            if acc >= 0.95:
                rating = "🟢 Excellent"
            elif acc >= 0.90:
                rating = "🟡 Very Good"
            elif acc >= 0.85:
                rating = "🟠 Good"
            else:
                rating = "🔴 Fair"

            print(f"{model_name:<25} {acc*100:>6.2f}%       [{bar}] {rating}")

        print("\n🏆 BEST MODEL: " + max(self.model_accuracies, key=self.model_accuracies.get))
        print("✅ All models ready for predictions!")

        return self.ensemble_model

    def save_models(self, filename='disease_predictor_multimodel.pkl'):
        """Save all trained models"""
        model_data = {
            'model_rf': self.model_rf,
            'model_svm': self.model_svm,
            'model_gb': self.model_gb,
            'ensemble_model': self.ensemble_model,
            'symptom_list': self.symptom_list,
            'symptom_to_original': self.symptom_to_original,
            'disease_symptom_map': self.disease_symptom_map,
            'model_accuracies': self.model_accuracies
        }
        joblib.dump(model_data, filename)
        print(f"\n💾 All models saved to: {filename}")

    def get_user_symptoms(self):
        """Interactive symptom input"""
        self.print_header("🩺 SYMPTOM INPUT", "═")

        print("\n📋 AVAILABLE SYMPTOMS (Sample - First 40)")
        print("─" * 80)

        for i in range(0, min(40, len(self.symptom_list)), 4):
            symptoms = self.symptom_list[i:i+4]
            row = "  ".join([f"• {s:20}" for s in symptoms])
            print(f"  {row}")

        if len(self.symptom_list) > 40:
            print(f"\n  ... and {len(self.symptom_list) - 40} more symptoms available")

        print("\n" + "─" * 80)
        print("💡 How to enter symptoms:")
        print("   • Type symptom name (e.g., 'fever', 'cough', 'headache')")
        print("   • Press Enter after each symptom")
        print("   • Type 'done' when finished")
        print("─" * 80)

        user_symptoms = []
        symptom_count = 1

        while True:
            symptom = input(f"\n🔹 Symptom {symptom_count}: ").strip().lower()

            if symptom == 'done':
                if not user_symptoms:
                    print("   ⚠️  Please enter at least one symptom!")
                    continue
                break

            if not symptom:
                continue

            matched = False
            best_matches = []

            for s in self.symptom_list:
                if symptom == s.lower():
                    user_symptoms.append(s)
                    print(f"   ✅ Added: {s}")
                    matched = True
                    symptom_count += 1
                    break
                elif symptom in s.lower():
                    best_matches.append(s)

            if not matched and best_matches:
                print(f"\n   🔍 Similar symptoms found:")
                for idx, match in enumerate(best_matches[:5], 1):
                    print(f"      {idx}. {match}")

                choice = input(f"\n   Select (1-{min(5, len(best_matches))}) or 'n': ").strip()

                if choice.isdigit() and 1 <= int(choice) <= len(best_matches):
                    selected = best_matches[int(choice)-1]
                    user_symptoms.append(selected)
                    print(f"   ✅ Added: {selected}")
                    symptom_count += 1

            elif not matched:
                print(f"   ❌ '{symptom}' not recognized")

        return user_symptoms

    def predict_with_all_models(self, symptoms):
        """Get predictions from all models and create consensus"""
        feature_vector = np.zeros((1, len(self.symptom_list)))

        for symptom in symptoms:
            symptom_lower = symptom.lower()
            if symptom_lower in self.symptom_list:
                idx = self.symptom_list.index(symptom_lower)
                feature_vector[0, idx] = 1

        # Get predictions from each model
        predictions_by_model = {}

        # Random Forest
        proba_rf = self.model_rf.predict_proba(feature_vector)[0]
        top_idx_rf = np.argmax(proba_rf)
        predictions_by_model['Random Forest'] = {
            'disease': self.model_rf.classes_[top_idx_rf],
            'confidence': float(proba_rf[top_idx_rf])
        }

        # SVM
        proba_svm = self.model_svm.predict_proba(feature_vector)[0]
        top_idx_svm = np.argmax(proba_svm)
        predictions_by_model['SVM'] = {
            'disease': self.model_svm.classes_[top_idx_svm],
            'confidence': float(proba_svm[top_idx_svm])
        }

        # Gradient Boosting
        proba_gb = self.model_gb.predict_proba(feature_vector)[0]
        top_idx_gb = np.argmax(proba_gb)
        predictions_by_model['Gradient Boosting'] = {
            'disease': self.model_gb.classes_[top_idx_gb],
            'confidence': float(proba_gb[top_idx_gb])
        }

        # Ensemble (Final decision)
        proba_ensemble = self.ensemble_model.predict_proba(feature_vector)[0]
        classes = self.ensemble_model.classes_
        top_indices = np.argsort(proba_ensemble)[::-1][:5]

        final_predictions = []
        for idx in top_indices:
            final_predictions.append({
                'disease': classes[idx],
                'confidence': float(proba_ensemble[idx])
            })

        return final_predictions, predictions_by_model, feature_vector

    def explain_prediction(self, disease_name, user_symptoms):
        """Generate explanation for prediction"""
        disease_symptoms = self.disease_symptom_map.get(disease_name, set())

        matching_symptoms = [s for s in user_symptoms if s.lower() in disease_symptoms]
        missing_symptoms = list(disease_symptoms - set([s.lower() for s in user_symptoms]))[:5]

        total_disease_symptoms = len(disease_symptoms)
        matched_count = len(matching_symptoms)
        match_percentage = (matched_count / total_disease_symptoms * 100) if total_disease_symptoms > 0 else 0

        return {
            'matching_symptoms': matching_symptoms,
            'missing_symptoms': missing_symptoms,
            'match_percentage': match_percentage,
            'total_disease_symptoms': total_disease_symptoms,
            'matched_count': matched_count
        }

    def get_medicine_info(self, disease_name):
        """Get medicine information"""
        disease_lower = disease_name.strip().lower()

        if disease_lower in DISEASE_MEDICINE_DATABASE:
            return DISEASE_MEDICINE_DATABASE[disease_lower]

        for key, info in DISEASE_MEDICINE_DATABASE.items():
            if disease_lower in key or key in disease_lower:
                return info

        return {
            'medicines': [
                {'name': 'Consult a doctor', 'type': 'Professional advice', 'dosage': 'Required'}
            ],
            'note': '⚠️ No specific data - Consult healthcare professional'
        }

    def format_medicine_table(self, disease_name):
        """Format medicine table"""
        info = self.get_medicine_info(disease_name)
        output = "\n" + "─" * 80 + "\n"
        output += "  💊 RECOMMENDED MEDICINES\n"
        output += "─" * 80 + "\n"

        medicines = info.get('medicines', [])

        if medicines:
            output += f"\n{'No.':<5}{'Medicine':<28}{'Type':<24}{'Dosage'}\n"
            output += "─" * 80 + "\n"

            for i, med in enumerate(medicines, 1):
                name = med.get('name', 'N/A')[:26]
                med_type = med.get('type', 'N/A')[:22]
                dosage = med.get('dosage', 'As prescribed')[:20]
                output += f"{i:<5}{name:<28}{med_type:<24}{dosage}\n"

        if 'alternatives' in info:
            output += f"\n📦 Alternatives: {', '.join(info['alternatives'])}\n"

        if 'note' in info:
            output += f"\n{info['note']}\n"

        output += "\n⚠️  IMPORTANT:\n"
        output += "   • Consult doctor before taking medication\n"
        output += "   • Do NOT self-medicate\n"
        output += "   • Follow prescribed dosage strictly\n"
        output += "─" * 80 + "\n"

        return output

    def get_disease_info(self, disease_name):
        """Get disease information"""
        info = {'disease': disease_name}

        try:
            desc_row = self.disease_descriptions[
                self.disease_descriptions['Disease'].str.strip().str.lower() == disease_name.strip().lower()
            ]
            if not desc_row.empty:
                info['description'] = desc_row.iloc[0]['Description']

            prec_row = self.disease_precautions[
                self.disease_precautions['Disease'].str.strip().str.lower() == disease_name.strip().lower()
            ]
            if not prec_row.empty:
                precautions = []
                for col in prec_row.columns:
                    if col.startswith('Precaution') and pd.notna(prec_row.iloc[0][col]):
                        prec = str(prec_row.iloc[0][col]).strip()
                        if prec:
                            precautions.append(prec)
                info['precautions'] = precautions

        except Exception:
            pass

        return info

    def display_results(self, final_predictions, predictions_by_model, user_symptoms, detailed_info=None):
        """Display comprehensive results with multi-model analysis"""
        self.print_header("🎯 MULTI-MODEL DIAGNOSIS RESULTS", "═")

        # Summary Box
        print("\n┌" + "─" * 78 + "┐")
        print(f"│  👤 Symptoms Analyzed: {len(user_symptoms):<53}│")
        print(f"│  🤖 Models Used: Random Forest + SVM + Gradient Boosting{' ' * 19}│")
        print(f"│  📊 Ensemble Prediction: Combined intelligence from 3 models{' ' * 13}│")
        print("└" + "─" * 78 + "┘")

        # Individual Model Predictions
        self.print_section("🤖 INDIVIDUAL MODEL PREDICTIONS")

        for model_name, pred in predictions_by_model.items():
            print(f"\n{model_name}:")
            print(f"   • Prediction: {pred['disease']}")
            print(f"   • Confidence: {pred['confidence']*100:.1f}%")
            print(f"   • Model Accuracy: {self.model_accuracies[model_name]*100:.2f}%")

        # Consensus Analysis
        self.print_section("🎯 MODEL CONSENSUS ANALYSIS")

        all_predicted_diseases = [pred['disease'] for pred in predictions_by_model.values()]
        unique_diseases = set(all_predicted_diseases)

        if len(unique_diseases) == 1:
            print("\n✅ UNANIMOUS AGREEMENT: All 3 models agree on the same disease!")
            print(f"   This increases confidence in the diagnosis.")
        elif len(unique_diseases) == 2:
            print("\n🟡 PARTIAL CONSENSUS: 2 models agree, 1 differs")
            print(f"   Ensemble uses weighted voting for final decision.")
        else:
            print("\n🔴 DIVERGENT PREDICTIONS: All 3 models predict different diseases")
            print(f"   Suggests complex case - doctor consultation recommended.")

        # Final Ensemble Prediction
        self.print_section("📊 FINAL ENSEMBLE PREDICTION (Top 5)")

        def get_status(conf):
            if conf >= 0.70: return " 🔴 HIGH"
            elif conf >= 0.40: return "🟡 MEDIUM"
            else: return " 🟢LOW"

        for i, pred in enumerate(final_predictions, 1):
            disease = pred['disease']
            confidence = pred['confidence'] * 100
            status = get_status(pred['confidence'])

            bar_length = int(confidence / 2)
            bar = '█' * bar_length + '░' * (50 - bar_length)

            print(f"\n{i}. {disease}")
            print(f"   Confidence: {confidence:.1f}% | Status: {status}")
            print(f"   [{bar}]")

        # Primary Diagnosis
        top_pred = final_predictions[0]
        confidence = top_pred['confidence']

        self.print_section(f"🏥 PRIMARY DIAGNOSIS: {top_pred['disease'].upper()}")
        print(f"\n📊 Final Confidence: {confidence*100:.1f}%")
        print(f"🎯 Reliability: {get_status(confidence)}")

        # Explanation
        self.print_section("🔍 WHY THIS DISEASE WAS PREDICTED")

        explanation = self.explain_prediction(top_pred['disease'], user_symptoms)

        print(f"\n📌 Analysis:")
        print(f"   • Disease symptoms in database: {explanation['total_disease_symptoms']}")
        print(f"   • Your matching symptoms: {explanation['matched_count']}")
        print(f"   • Match rate: {explanation['match_percentage']:.1f}%")

        if explanation['matching_symptoms']:
            print(f"\n✅ YOUR MATCHING SYMPTOMS:")
            for i, symptom in enumerate(explanation['matching_symptoms'], 1):
                print(f"   {i}. {symptom} ✓")

        if explanation['missing_symptoms']:
            print(f"\n⚪ COMMON SYMPTOMS YOU DON'T HAVE:")
            for i, symptom in enumerate(explanation['missing_symptoms'], 1):
                print(f"   {i}. {symptom}")

        # Low confidence warning
        if confidence < self.confidence_threshold:
            print("\n┌" + "─" * 78 + "┐")
            print("│  ⚠️  LOW CONFIDENCE - CONSULT DOCTOR IMMEDIATELY".center(78) + "│")
            print("└" + "─" * 78 + "┘")

        # Disease description
        if detailed_info and 'description' in detailed_info:
            self.print_section("📋 ABOUT THIS CONDITION")
            desc = detailed_info['description']
            words = desc.split()
            lines = []
            current_line = ""
            for word in words:
                if len(current_line) + len(word) + 1 <= 75:
                    current_line += word + " "
                else:
                    lines.append(current_line.strip())
                    current_line = word + " "
            if current_line:
                lines.append(current_line.strip())

            for line in lines:
                print(f"  {line}")

        # Medicine recommendations
        print(self.format_medicine_table(top_pred['disease']))

        # Precautions
        if detailed_info and 'precautions' in detailed_info and detailed_info['precautions']:
            self.print_section("🛡️  RECOMMENDED PRECAUTIONS")
            for i, prec in enumerate(detailed_info['precautions'], 1):
                print(f"  {i}. ✓ {prec}")

        # Disclaimer
        print("\n" + "═" * 80)
        print("⚕️  MEDICAL DISCLAIMER".center(80))
        print("═" * 80)
        print("""
  ⚠️  Educational purposes only - NOT medical advice
  ❌ NOT a substitute for professional diagnosis
  ✅ Always consult qualified healthcare providers
  🚨 Emergency? Contact medical services immediately
""")
        print("═" * 80)

    def run(self):
        """Main workflow"""
        self.print_header("🏥 MULTI-MODEL DISEASE PREDICTION", "═")

        user_symptoms = self.get_user_symptoms()

        if not user_symptoms:
            print("\n❌ No symptoms entered.")
            return

        print("\n" + "═" * 80)
        print("⏳ ANALYZING WITH MULTIPLE ML MODELS...".center(80))
        print("═" * 80)
        print(f"\n📋 Your Symptoms ({len(user_symptoms)}):")
        for i, s in enumerate(user_symptoms, 1):
            print(f"   {i}. {s}")

        print("\n⚙️  Running analysis:")
        print("   🌲 Random Forest analyzing...")
        print("   🔵 SVM processing...")
        print("   📈 Gradient Boosting evaluating...")
        print("   🎯 Ensemble combining results...")

        final_predictions, predictions_by_model, feature_vector = self.predict_with_all_models(user_symptoms)
        detailed_info = self.get_disease_info(final_predictions[0]['disease'])

        self.display_results(final_predictions, predictions_by_model, user_symptoms, detailed_info)


def main():
    """Main function"""

    print("\n" + "═" * 80)
    print("🏥 MULTI-MODEL DISEASE PREDICTION SYSTEM".center(80))
    print("═" * 80)
    print("""
    🤖 3 ML Models: Random Forest + SVM + Gradient Boosting
    🎯 Ensemble Intelligence for Maximum Accuracy
    💊 Comprehensive Medicine Recommendations
    🔍 Detailed Explanations for Every Prediction

    Version 4.0 - Multi-Model Enhanced Edition
""")
    print("═" * 80)

    system = MultiModelDiseasePrediction()

    if not system.load_data():
        return

    if system.preprocess_data() is None:
        return

    system.train_models()
    system.save_models()

    while True:
        system.run()

        print("\n" + "═" * 80)
        choice = input("\n🔄 Another prediction? (yes/no): ").strip().lower()
        if choice not in ['yes', 'y']:
            break

    print("\n" + "═" * 80)
    print("✅ Thank you for using the system!".center(80))
    print("💚 Stay healthy!".center(80))
    print("═" * 80 + "\n")


if __name__ == "__main__":
    main()


════════════════════════════════════════════════════════════════════════════════
                    🏥 MULTI-MODEL DISEASE PREDICTION SYSTEM                     
════════════════════════════════════════════════════════════════════════════════

    🤖 3 ML Models: Random Forest + SVM + Gradient Boosting
    🎯 Ensemble Intelligence for Maximum Accuracy
    💊 Comprehensive Medicine Recommendations
    🔍 Detailed Explanations for Every Prediction

    Version 4.0 - Multi-Model Enhanced Edition

════════════════════════════════════════════════════════════════════════════════

════════════════════════════════════════════════════════════════════════════════
                               📂 LOADING DATASETS                               
════════════════════════════════════════════════════════════════════════════════

⏳ Loading Training.csv... ✅ [4920 records]
⏳ Loading symptom_Description.csv... ✅ [41 diseases]
⏳ Loading symptom_precaution.csv... ✅ [41 precautions]

✅ All required datasets lo